In [6]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.tools as tls
import time
import plotly
import glob
import os

In [51]:
plotly.offline.init_notebook_mode(connected=True) 

In [52]:
gt_path = '/raid/data/Datasets/PublicDatasets/KITTI/kitti-raw-uncompressed/2011_09_26/2011_09_26_drive_0009_sync/oxts/data'
prediction_path = '/home/trn_ak/git_clones/models/research/vid2depth/inference_validation_egomotion'

In [53]:
#data_gt=pd.read_csv(r'/home/trn_ak/git_clones/models/research/vid2depth/plan.txt',sep= '\t',header=None)
data_delta=pd.read_csv('/home/trn_ak/git_clones/models/research/vid2depth/inference_validation_egomotion/inference.txt',sep= '\t',header=None)

for gt_file in sorted(os.listdir(gt_path)):
        with open(os.path.join(gt_path, gt_file)) as gt_pose_file:
            data_gt=pd.read_csv(os.path.join(gt_path, gt_file),sep= '\t',header=None)
            #print(data_gt[0][0])
    #data_pred = data_gt + data_delta
    
#print(data_delta[0][0])

In [57]:
import numpy as np
import os
#import cv, cv2
import argparse
import tensorflow as tf
from absl import logging
import csv
#from evaluation_utils import *

#parser = argparse.ArgumentParser(description='Evaluation on the KITTI dataset')
#parser.add_argument('--split',               type=str,   help='data split, kitti or eigen',         required=True)
#parser.add_argument('--prediction_path', type=str,   help='path to estimated disparities',      required=True)
#parser.add_argument('--gt_path',             type=str,   help='path to ground truth disparities',   required=True)
#parser.add_argument('--min_threshold',           type=float, help='minimum threshold for depth/pose evaluation',        default=1e-3)
#parser.add_argument('--max_threshold',           type=float, help='maximum threshold for depth/pose evaluation',        default=80)
#parser.add_argument('--eigen_crop',                      help='if set, crops according to Eigen NIPS14',   action='store_true')

#args = parser.parse_args()
gfile = tf.gfile
split = 'kitti'

if __name__ == '__main__':

    if split == 'kitti':
        NUM_SAMPLES = 447
        
        if os.path.exists(prediction_path) and os.path.exists(gt_path):
            with open(os.path.join(prediction_path, 'inference.txt')) as pred_file:
                                
                    pred_reader = csv.reader(pred_file, delimiter=' ')
                    row_count = 0 
                    gt_prev_pose = []
                    ate_all = []
                    gt_all = []
                    pred_all = []
                    
                    for pred_row, gt_file in zip(pred_reader, sorted(os.listdir(gt_path))):
                        with open(pred_row[0]) as pred_pose_file, open(os.path.join(gt_path, gt_file)) as gt_pose_file:

                            pred_pose_reader = csv.reader(pred_pose_file, delimiter=' ')
                            gt_pose_reader = csv.reader(gt_pose_file, delimiter=' ')                            
                            #    DEBUG: To print the no. of rows in the file
                            #    a = sum(1 for line in gt_pose_file)

                            for delta_pose, gt_pose in zip(pred_pose_reader, gt_pose_reader):
                                gt_curr_pose = np.array([float(value) for value in gt_pose[0:6]])
                                delta_pose = np.array([float(value) for value in delta_pose])
#                                gt_curr_pose = [float(value) for value in gt_pose[0:6]]
#                                delta_pose = [float(value) for value in delta_pose]

                                if row_count == 0:
                                    gt_prev_pose = gt_curr_pose
                                else:
#                                    pred_pose = gt_prev_pose + delta_pose
                                    pred_pose = np.add(gt_prev_pose, delta_pose)
                                    scale = np.sum(gt_curr_pose * pred_pose)/np.sum(pred_pose ** 2)
                                    alignment_error = pred_pose * scale - gt_curr_pose 
#                                    rmse = np.sqrt(np.sum(alignment_error ** 2)))/len()
                                    rmse = np.sqrt(np.sum(alignment_error ** 2))/ NUM_SAMPLES
                                    ate_all.append(rmse)
                                    gt_prev_pose = gt_curr_pose
                                    gt_all.append(gt_curr_pose)
                                    pred_all.append(pred_pose)
                                   
                   
#                                    print("ROW Count {}".format(row_count))
#                                    print("gt_curr_pose : {}".format(gt_curr_pose[0]))
#                                    print("pred_pose lat: {}".format(pred_pose[0]))

#
                            row_count += 1
                    scatter_plot(pred_all, gt_all)

                    ate_all = np.array(ate_all)
                    print("ATE mean: {}, std: {}".format(np.mean(ate_all), np.std(ate_all)))


ATE mean: 5.07193671815e-05, std: 1.64581257613e-05


In [56]:
def scatter_plot(pred_pose, gt_pose):
    trace_gt = go.Scatter(x = gt_pose[0],
                          y = gt_pose[1], 
                          mode = 'lines+markers', 
                          name= 'Ground Truth',
                          line=dict(width= 2.5),
                          marker=dict(
                                  size=5.5,
                             ))

    trace_pred = go.Scatter(x = pred_pose[0],
                          y = pred_pose[1], 
                          mode = 'lines+markers', 
                          name= 'Prediction',
                          line=dict(width= 1.5),
                        opacity=0.7
                            ,
                          marker=dict(
                                  size=3.5,
                              color='Red'
                             ))

    data = [trace_gt, trace_pred]

    layout= go.Layout(
        title= 'KITTI Ground Truth vs vid2depth Pred',
        hovermode= 'closest',
        xaxis= dict(
            title= 'Latitude',
            ticklen= 5,
            zeroline= False,
            gridwidth= 2,
        ),
        yaxis=dict(
            title= 'Longitude',
            ticklen= 5,
            zeroline= True,
            gridwidth= 2,
        ),

        showlegend= True
    )
    fig= go.Figure(data=data, layout=layout)

    # Plot in a different tab!
    #plotly.offline.plot(fig, filename='vid2depth.html')   
    #plotly.plot(fig, filename='vid2depth.html')

    plotly.offline.iplot(fig, filename='Agv_pathVspose.html')   #To plot within the noteook
